Group 25
Shaleen Malik(12041360), Prarabdh Shukla(12041090), Vidhi Mittal(12041730), Vanisha Agrawal(12041670)

## Imports

In [ ]:
!pip install datasets

In [ ]:
!wget https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStories_all_data.tar.gz

--2024-03-27 05:45:14--  https://huggingface.co/datasets/roneneldan/TinyStories/resolve/main/TinyStories_all_data.tar.gz
Resolving huggingface.co (huggingface.co)... 3.163.189.37, 3.163.189.114, 3.163.189.74, ...
Connecting to huggingface.co (huggingface.co)|3.163.189.37|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/42/7f/427f7497b6c6596c18b46d5a72e61364fcad12aa433c60a0dbd4d344477b9d81/26cf7605aca15bc4ea6fa637256400d9d01317b28ed296172b2d1dd160cd7699?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27TinyStories_all_data.tar.gz%3B+filename%3D%22TinyStories_all_data.tar.gz%22%3B&response-content-type=application%2Fgzip&Expires=1711777514&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMTc3NzUxNH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy80Mi83Zi80MjdmNzQ5N2I2YzY1OTZjMThiNDZkNWE3MmU2MTM2NGZjYWQxMmFhNDMzYzYwYTBkYmQ0ZDM0NDQ3N2I5ZDgxLzI2Y2Y3

In [ ]:
!tar -xzf /content/TinyStories_all_data.tar.gz -C TinyStories

tar: TinyStories: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import os
import glob
import json
import torch
import numpy as np
import torch
import torch.nn as nn
from torch.nn import functional as F

## Data Preprocessing

### Getting data


In [ ]:
foo = open("/content/TinyStories-valid.txt", "r")
content = foo.readlines()
con_str = ''
for c in content:
  con_str += c

stories = con_str.split("<|endoftext|>\n")

In [ ]:
stories[100]

'Once upon a time, there was a little boy named Timmy. Timmy loved to eat peaches. One day, he saw a peach on the ground and picked it up. But the peach was hard and not ripe yet. Timmy was sad because he couldn\'t eat it.\nTimmy\'s mom saw him and said, "Don\'t worry, Timmy. The peach will be ripe soon. Just wait a little longer." Timmy waited and waited, and finally, the peach became soft and juicy. He was so happy that he could finally eat it.\nFrom that day on, Timmy learned that good things come to those who wait. He also learned that sometimes things might seem hard or sharp, but with a little patience, they can become sweet and delicious, just like his peach.\n'

In [ ]:
text = "\n".join(stories)

In [ ]:
len(text)

19147122

In [ ]:
print(text[:100])

 Spot. Spot saw the shiny car and said, "Wow, Kitty, your car is so bright and clean!" Kitty smiled 


### Vocabulary

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$&'()*+,-./0123456789:;<?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz­´éñ ​–—‘’“”…🎓
96


In [ ]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = {i:ch for i,ch in enumerate(chars)}

In [ ]:
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[x] for x in l])

In [ ]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, type(data))
print(data[:100])

torch.Size([19147122]) <class 'torch.Tensor'>
tensor([ 1, 48, 71, 70, 75, 14,  1, 48, 71, 70, 75,  1, 74, 56, 78,  1, 75, 63,
        60,  1, 74, 63, 64, 69, 80,  1, 58, 56, 73,  1, 56, 69, 59,  1, 74, 56,
        64, 59, 12,  1,  3, 52, 70, 78, 12,  1, 40, 64, 75, 75, 80, 12,  1, 80,
        70, 76, 73,  1, 58, 56, 73,  1, 64, 74,  1, 74, 70,  1, 57, 73, 64, 62,
        63, 75,  1, 56, 69, 59,  1, 58, 67, 60, 56, 69,  2,  3,  1, 40, 64, 75,
        75, 80,  1, 74, 68, 64, 67, 60, 59,  1])


### Data Preparation

In [ ]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8

In [ ]:
train_data[:block_size+1]

tensor([ 1, 48, 71, 70, 75, 14,  1, 48, 71])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
x,y

(tensor([ 1, 48, 71, 70, 75, 14,  1, 48]),
 tensor([48, 71, 70, 75, 14,  1, 48, 71]))

In [ ]:
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("ctx ", context, "target", target)

ctx  tensor([1]) target tensor(48)
ctx  tensor([ 1, 48]) target tensor(71)
ctx  tensor([ 1, 48, 71]) target tensor(70)
ctx  tensor([ 1, 48, 71, 70]) target tensor(75)
ctx  tensor([ 1, 48, 71, 70, 75]) target tensor(14)
ctx  tensor([ 1, 48, 71, 70, 75, 14]) target tensor(1)
ctx  tensor([ 1, 48, 71, 70, 75, 14,  1]) target tensor(48)
ctx  tensor([ 1, 48, 71, 70, 75, 14,  1, 48]) target tensor(71)


In [ ]:
torch.manual_seed(1337)

In [ ]:
batch_size = 4

### Loading data

In [ ]:
# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [ ]:
xb, yb = get_batch('train')

In [ ]:
device= 'cuda'

In [ ]:
yb

tensor([[62, 14,  0,  0, 44, 69, 58, 60],
        [60,  1, 76, 71, 70, 69,  1, 56],
        [ 1, 74, 64, 67, 67, 80, 14,  1],
        [60,  1, 78, 60, 69, 75,  1, 70]])

In [ ]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b][:t+1]
        target = yb[b][t]
        print(context, "-----", target)

tensor([69]) ----- tensor(62)
tensor([69, 62]) ----- tensor(14)
tensor([69, 62, 14]) ----- tensor(0)
tensor([69, 62, 14,  0]) ----- tensor(0)
tensor([69, 62, 14,  0,  0]) ----- tensor(44)
tensor([69, 62, 14,  0,  0, 44]) ----- tensor(69)
tensor([69, 62, 14,  0,  0, 44, 69]) ----- tensor(58)
tensor([69, 62, 14,  0,  0, 44, 69, 58]) ----- tensor(60)
tensor([58]) ----- tensor(60)
tensor([58, 60]) ----- tensor(1)
tensor([58, 60,  1]) ----- tensor(76)
tensor([58, 60,  1, 76]) ----- tensor(71)
tensor([58, 60,  1, 76, 71]) ----- tensor(70)
tensor([58, 60,  1, 76, 71, 70]) ----- tensor(69)
tensor([58, 60,  1, 76, 71, 70, 69]) ----- tensor(1)
tensor([58, 60,  1, 76, 71, 70, 69,  1]) ----- tensor(56)
tensor([70]) ----- tensor(1)
tensor([70,  1]) ----- tensor(74)
tensor([70,  1, 74]) ----- tensor(64)
tensor([70,  1, 74, 64]) ----- tensor(67)
tensor([70,  1, 74, 64, 67]) ----- tensor(67)
tensor([70,  1, 74, 64, 67, 67]) ----- tensor(80)
tensor([70,  1, 74, 64, 67, 67, 80]) ----- tensor(14)
tensor(

In [ ]:
vocab_size

96

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1137)

## Defining Model




### Baseline MOE

Gating function, returns top k experts

In [ ]:
class MoeLayer(nn.Module):
    def __init__(self, experts, gate, k=1):
        super().__init__()
        assert len(experts) > 0
        self.experts = nn.ModuleList(experts)
        self.gate = gate
        self.k = k

    def forward(self, inputs: torch.Tensor):
        inputs_squashed = inputs.view(-1, inputs.shape[-1])
        gate_logits = self.gate(inputs_squashed)
        weights, selected_experts = torch.topk(
            gate_logits, self.k
        )
        weights = nn.functional.softmax(
            weights,
            dim=1,
            dtype=torch.float,
        ).type_as(inputs)
        results = torch.zeros_like(inputs_squashed)
        for i, expert in enumerate(self.experts):
            batch_idx, nth_expert = torch.where(selected_experts == i)
            results[batch_idx] += weights[batch_idx, nth_expert, None] * expert(
                inputs_squashed[batch_idx]
            )
        return results.view_as(inputs)

Model

In [ ]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias = False)
        self.query = nn.Linear(n_embed, head_size, bias = False)
        self.value = nn.Linear(n_embed, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MulitHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x =  torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.dropout(self.proj(x))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4* n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
         nn.Dropout(dropout))

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embed, n_head, num_experts=4):
        super().__init__()
        self.sa_head= MulitHeadAttention(n_head, n_embed//n_head)
        self.ffw = MoeLayer(
            experts=[FeedForward(n_embed) for _ in range(num_experts)],
            gate=nn.Linear(n_embed, num_experts, bias=False),
        )

#       self.ffw=  FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_head(self.ln1(x))
        x = x+self.ffw(self.ln2(x))
        return x


class Transformer(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embed, device=device)
        self.position_embedding_table = nn.Embedding(block_size, n_embed, device=device)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embed, vocab_size)


    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = token_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokes):
        for _ in range(max_new_tokes):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -132089440, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

## Model Training

In [ ]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_embed = 384
n_head = 6
n_layer = 6
dropout = 0.0
low_rank = True
# ------------

In [ ]:
model = Transformer()
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X = X.to(device)
            Y = Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
device

'cpu'

In [ ]:
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.requires_grad = True
    loss.backward()
    optimizer.step()

step 0: train loss 4.9518, val loss 4.9487
step 100: train loss 4.9516, val loss 4.9505
step 200: train loss 4.9507, val loss 4.9483
step 300: train loss 4.9515, val loss 4.9496
step 400: train loss 4.9507, val loss 4.9490
step 500: train loss 4.9510, val loss 4.9495
step 600: train loss 4.9509, val loss 4.9498
step 700: train loss 4.9511, val loss 4.9496
step 800: train loss 4.9516, val loss 4.9498
step 900: train loss 4.9508, val loss 4.9493
step 999: train loss 4.9508, val loss 4.9490


In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

32089440


Time taken = 36min 55 sec \
Number of parameters = 32089440 \
Validation loss = 4.94 \

## With Low Rank Adaptation

In [ ]:
!pip install loralib

In [ ]:
import loralib as lora

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = lora.Linear(n_embed, head_size, r=16)
        # nn.Linear(n_embed, head_size, bias = False)
        self.query = lora.Linear(n_embed, head_size, r=16)
        # nn.Linear(n_embed, head_size, bias = False)
        self.value = lora.Linear(n_embed, head_size, r=16)
        # nn.Linear(n_embed, head_size, bias = False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

class MulitHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = lora.Linear(n_embed, n_embed, r=16)
        #  nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x =  torch.cat([head(x) for head in self.heads], dim=-1)
        out = self.dropout(self.proj(x))
        return out


class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            lora.Linear(n_embed, 4* n_embed, r=16),
            nn.ReLU(),
            lora.Linear(4 * n_embed, n_embed, r=16),
         nn.Dropout(dropout))

    def forward(self, x):
        return self.net(x)


class Block(nn.Module):
    def __init__(self, n_embed, n_head, num_experts=4):
        super().__init__()
        self.sa_head= MulitHeadAttention(n_head, n_embed//n_head)
        self.ffw = MoeLayer(
            experts=[FeedForward(n_embed) for _ in range(num_experts)],
            gate=lora.Linear(n_embed, num_experts, r=16),
        )

#       self.ffw=  FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa_head(self.ln1(x))
        x = x+self.ffw(self.ln2(x))
        return x


class Transformer(nn.Module):
    def __init__(self):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, n_embed, device=device)
        self.position_embedding_table = nn.Embedding(block_size, n_embed, device=device)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.lm_head = lora.Linear(n_embed, vocab_size, r=16)


    def forward(self, idx, targets=None):
        B, T = idx.shape

        token_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = token_emb + pos_emb
        x = self.blocks(x)
        logits = self.lm_head(x)
        if targets == None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokes):
        for _ in range(max_new_tokes):
            idx_cond = idx[:, -block_size:]
            logits, loss = self(idx_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples = 1)
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx

In [ ]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_embed = 384
n_head = 6
n_layer = 6
dropout = 0.0
low_rank = True
# ------------

In [ ]:
model = Transformer()
lora.mark_only_lora_as_trainable(model)
optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate)

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            X = X.to(device)
            Y = Y.to(device)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [ ]:
device

'cuda'

In [ ]:
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(),lr=1e-4)

In [ ]:
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % 100 == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 4.9131, val loss 4.9126
step 100: train loss 3.9347, val loss 3.9342
step 200: train loss 3.1982, val loss 3.1951
step 300: train loss 3.0063, val loss 2.9995
step 400: train loss 2.8888, val loss 2.8824
step 500: train loss 2.8037, val loss 2.7952
step 600: train loss 2.7333, val loss 2.7254
step 700: train loss 2.6803, val loss 2.6713
step 800: train loss 2.6309, val loss 2.6251
step 900: train loss 2.5914, val loss 2.5806
step 999: train loss 2.5533, val loss 2.5467


In [ ]:
pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(pytorch_total_params)

2367360


Time taken = 22min 47 sec \
Number of parameters = 2367360 \
Validation loss = 2.54

## Testing model functioning

In [ ]:
d = 'once upon a time there was a '
x = torch.tensor(encode(d), dtype = torch.long,device=device).unsqueeze(0)
print(decode(model.generate(x, max_new_tokes=500)[0].tolist()))


once upon a time there was a girl named Lily. She liked to pretend the ambulance catles: "Can I be careful when you catch the ambulance?" Lily was careful and protected the amaze. After her a while, her parents came by their amazing cat with a good when put in the ground. Lily realized she wanted some chest and her mom told her those about playing with the big maze they had splashed and slapped the magic toy. 
After that, she decided to open it in the living room and told her that they took the majar, she heard a voice. Gra


In [ ]:
d = 'One day'
x = torch.tensor(encode(d), dtype = torch.long,device=device).unsqueeze(0)
print(decode(model.generate(x, max_new_tokes=500)[0].tolist()))


One day, she was playing with her toys.
While playing, she saw a big pin on the yard. It wanted to safe again looking at it. Lily wanted to help her mother, but so she talked about the pink jump and better.
"Wow, Lily!" said the pink is a pink. "What rainbows are!"
"The next was safe," said the dog's happy steps. "Thank you, Lily," said Lily.
"I found a big ice-cream," the pink grew drain.
"Yes, let's play!" said Lily.
But the warn's white was rough. "My tooughs are looking in the sky!" Lily smiled. Sh


## Post Training Quantization

In [ ]:
!pip install transformers
!pip install accelerate
!git clone https://github.com/mit-han-lab/smoothquant.git

In [ ]:
import torch
from transformers.models.opt.modeling_opt import (
    OPTAttention,
    OPTDecoderLayer,
    OPTForCausalLM,
)
from transformers import GPT2Tokenizer
# from content.smoothquant import smoothquant
from smoothquant.smoothquant.smooth import smooth_lm
from smoothquant.smoothquant.fake_quant import W8A8Linear, quantize_opt
from tqdm import tqdm
import time

In [ ]:
class Evaluator:
    def __init__(self, dataset, tokenizer, device):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.device = device
        print("hello")

        # tokenize the dataset
        def tokenize_function(examples):
            example = self.tokenizer(examples["text"])
            return example

        self.dataset = self.dataset.map(tokenize_function, batched=True)
        self.dataset.set_format(type="torch", columns=["input_ids"])
        print("hello2")

    @torch.no_grad()
    def evaluate(self, model):
        model.to(self.device)
        model.eval()
        # The task is to predict the last word of the input.
        total, hit = 0, 0
        idx_count=0
        print("hlos")
        for batch in tqdm(self.dataset):
            print(idx_count)
            input_ids = batch["input_ids"].to(self.device).unsqueeze(0)
            # print(l)
            label = input_ids[:, -1]
            outputs = model(input_ids)
            print(label.shape)
            last_token_logits = outputs.logits[:, -2, :]
            pred = last_token_logits.argmax(dim=-1)
            total += label.size(0)
            hit += (pred == label).sum().item()
            idx_count+=1

        acc = hit / total
        return acc

In [ ]:
from datasets import load_dataset
tokenizer = GPT2Tokenizer.from_pretrained("facebook/opt-13b")
dataset = load_dataset("lambada", split="validation[:1000]")
evaluator = Evaluator(dataset, tokenizer, "cuda")

OPT_FP16 Model

In [ ]:
!mkdir offload

In [ ]:
model_fp16 = OPTForCausalLM.from_pretrained(
    "facebook/opt-13b", torch_dtype=torch.float16, device_map="auto", offload_folder="/content/offload"
)

In [ ]:
start_time=time.time()
acc_fp16 = evaluator.evaluate(model_fp16)
end_time=time.time()
print(f"Original model (fp16) accuracy: {acc_fp16}")
print("Time taken by original model (fp16): ", end_time-start_time, "seconds" )

Original model (fp16) accuracy: 0.769
Time taken by original model (fp16):  56.145999908447266 seconds


OPT_INT8 Model

In [ ]:
!python "/content/smoothquant/examples/generate_act_scales.py"

In [ ]:
act_scales = torch.load("../act_scales/opt-13b.pt")
smooth_lm(model_fp16, act_scales, 0.5)
model_smoothquant_w8a8 = quantize_opt(model_fp16)

In [ ]:
start_time=time.time()
acc_smoothquant_w8a8 = evaluator.evaluate(model_smoothquant_w8a8)
end_time=time.time()
print(f"SmoothQuant W8A8 quantized model accuracy: {acc_smoothquant_w8a8}")
print("Time taken by SmoothQuant W8A8 quantized model (INT8): ", end_time-start_time, "seconds" )

SmoothQuant W8A8 quantized model accuracy: 0.775
Time taken by SmoothQuant W8A8 quantized model (INT8):  35.66000008583069 seconds


# Metaflow Notebook Example

## Install wandb and metaflow deps

In [ ]:
!pip install -Uqqq metaflow fastcore wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.2/82.2 kB 10.1 MB/s eta 0:00:00


In [ ]:
!wandb login

wandb: Currently logged in as: megatruong (use `wandb login --relogin` to force relogin)


In [ ]:
%%writefile test.py
import os

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import wandb
from wandb.integration.metaflow import wandb_log

from metaflow import FlowSpec, Parameter, step

# os.environ["WANDB_SILENT"] = "true"
os.environ["METAFLOW_USER"] = "test_user"


@wandb_log(datasets=True, models=True, others=True, settings=wandb.Settings(project='metaflow_integration'))
class WandbExampleFlowDecoClass(FlowSpec):
    # Not obvious how to support metaflow.IncludeFile
    seed = Parameter("seed", default=1337)
    test_size = Parameter("test_size", default=0.2)
    raw_data = Parameter(
        "raw_data",
        default="https://gist.githubusercontent.com/tijptjik/9408623/raw/b237fa5848349a14a14e5d4107dc7897c21951f5/wine.csv",
        help="path to the raw data",
    )

    @step
    def start(self):
        self.raw_df = pd.read_csv(self.raw_data)
        self.next(self.split_data)

    @step
    def split_data(self):
        X = self.raw_df.drop("Wine", axis=1)
        y = self.raw_df[["Wine"]]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            X, y, test_size=self.test_size, random_state=self.seed
        )
        self.next(self.train)

    @step
    def train(self):
        self.clf = RandomForestClassifier(random_state=self.seed)
        self.clf.fit(self.X_train, self.y_train)
        self.next(self.end)

    @step
    def end(self):
        self.preds = self.clf.predict(self.X_test)
        self.accuracy = accuracy_score(self.y_test, self.preds)


if __name__ == "__main__":
    WandbExampleFlowDecoClass()

Overwriting test.py


In [ ]:
!python test.py --no-pylint run

Metaflow 2.3.5 executing WandbExampleFlowDecoClass for user:test_user
Validating your flow...
    The graph looks good!
2021-08-26 21:29:33.893 Workflow starting (run-id 1630013373888157):
2021-08-26 21:29:33.905 [1630013373888157/start/1 (pid 1455)] Task is starting.
2021-08-26 21:29:36.358 [1630013373888157/start/1 (pid 1455)] wandb: Currently logged in as: megatruong (use `wandb login --relogin` to force relogin)
2021-08-26 21:29:38.964 [1630013373888157/start/1 (pid 1455)] wandb: Tracking run with wandb version 0.12.1
2021-08-26 21:29:38.964 [1630013373888157/start/1 (pid 1455)] wandb: Syncing run splendid-puddle-1
2021-08-26 21:29:38.964 [1630013373888157/start/1 (pid 1455)] wandb: ⭐️ View project at https://wandb.ai/megatruong/metaflow_integration
2021-08-26 21:29:38.965 [1630013373888157/start/1 (pid 1455)] wandb: 🚀 View run at https://wandb.ai/megatruong/metaflow_integration/runs/2vgelzt0
2021-08-26 21:29:38.965 [1630013373888157/start/1 (pid 1455)] wandb: Run data is saved loc